# Imports

In [19]:
import pandas as pd
import requests
from datetime import datetime as dt

In [20]:
import sys
ROOT = '../'
sys.path.append(ROOT)  # Add the root folder to the sys.path

# Import the modules
from config import *

# Reload the configuration
from importlib import reload
reload(sys.modules['config'])

# Import the reloaded modules
from config import *

# Fear and Greed Index

In [21]:
# Set the limit
limit = 0 # All data available

# URL for the API
url = f"https://api.alternative.me/fng/?limit={limit}&format=json"

# Execute the request
response = requests.get(url)

fng_raw = None

# Check for a successful response
if response.status_code == 200:
    # Extract the JSON data
    data = response.json()
    
    # Extract the data from the JSON
    fng_raw = data['data']
    
    # Crea un DataFrame pandas
    fng_raw = pd.DataFrame(fng_raw)
else:
    print("Errore nella richiesta all'API:", response.status_code)

In [22]:
fng_raw

,value,value_classification,timestamp,time_until_update
0,61,Greed,1722384000,28781
1,67,Greed,1722297600,NaN
2,74,Greed,1722211200,NaN
3,71,Greed,1722124800,NaN
4,72,Greed,1722038400,NaN
...,...,...,...,...
2365,11,Extreme Fear,1517788800,NaN
2366,24,Extreme Fear,1517702400,NaN
2367,40,Fear,1517616000,NaN
2368,15,Extreme Fear,1517529600,NaN


In [23]:
fng = fng_raw.copy()

# Delete timestamp and time_until columns
fng = fng.drop(columns=['time_until_update'])

# Rename columns
fng = fng.rename(columns={'value': 'fng_value', 'value_classification': 'fng_value_classification'})

# Convert fng_value to float 72 -> 0.72
fng['fng_value'] = fng['fng_value'].astype(float)/100

# Convert timestamp to datetime
fng['timestamp'] = pd.to_datetime(fng['timestamp'], unit='s')
fng


C:\Users\danil\AppData\Local\Temp\ipykernel_38432\2134796205.py:13: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  fng['timestamp'] = pd.to_datetime(fng['timestamp'], unit='s')


,fng_value,fng_value_classification,timestamp
0,0.61,Greed,2024-07-31
1,0.67,Greed,2024-07-30
2,0.74,Greed,2024-07-29
3,0.71,Greed,2024-07-28
4,0.72,Greed,2024-07-27
...,...,...,...
2365,0.11,Extreme Fear,2018-02-05
2366,0.24,Extreme Fear,2018-02-04
2367,0.40,Fear,2018-02-03
2368,0.15,Extreme Fear,2018-02-02


In [24]:
# Rename value_clasification values using the following mapping
value_classification_mapping = {
    'Extreme Fear': 0,
    'Fear': 1,
    'Neutral': 2,
    'Greed': 3,
    'Extreme Greed': 4
}

fng['fng_value_classification'] = fng['fng_value_classification'].map(value_classification_mapping)
fng

,fng_value,fng_value_classification,timestamp
0,0.61,3,2024-07-31
1,0.67,3,2024-07-30
2,0.74,3,2024-07-29
3,0.71,3,2024-07-28
4,0.72,3,2024-07-27
...,...,...,...
2365,0.11,0,2018-02-05
2366,0.24,0,2018-02-04
2367,0.40,1,2018-02-03
2368,0.15,0,2018-02-02


In [25]:
# For each fng_value, use the following mapping to determine the sentiment
# fng_value < 0.4: negative
# 0.4 <= fng_value < 0.6: neutral
# fng_value >= 0.6: positive

fng['fng_sentiment'] = 'negative'
fng.loc[fng['fng_value'] >= 0.4, 'fng_sentiment'] = 'neutral'
fng.loc[fng['fng_value'] >= 0.6, 'fng_sentiment'] = 'positive'
fng

,fng_value,fng_value_classification,timestamp,fng_sentiment
0,0.61,3,2024-07-31,positive
1,0.67,3,2024-07-30,positive
2,0.74,3,2024-07-29,positive
3,0.71,3,2024-07-28,positive
4,0.72,3,2024-07-27,positive
...,...,...,...,...
2365,0.11,0,2018-02-05,negative
2366,0.24,0,2018-02-04,negative
2367,0.40,1,2018-02-03,neutral
2368,0.15,0,2018-02-02,negative


In [26]:
# Trim the data to the specified date range
fng = fng[(fng["timestamp"] >= start_date) & (fng["timestamp"] <= end_date)]
fng

,fng_value,fng_value_classification,timestamp,fng_sentiment
0,0.61,3,2024-07-31,positive
1,0.67,3,2024-07-30,positive
2,0.74,3,2024-07-29,positive
3,0.71,3,2024-07-28,positive
4,0.72,3,2024-07-27,positive
...,...,...,...,...
2365,0.11,0,2018-02-05,negative
2366,0.24,0,2018-02-04,negative
2367,0.40,1,2018-02-03,neutral
2368,0.15,0,2018-02-02,negative


In [27]:
# Save the data to a CSV file
output_file = os.path.join(ROOT, INDEXES_DATASET_PATH, 'fng_daily.csv')
fng.to_csv(output_file, index=False)

In [ ]:
# # Set the timestamp as the index
# fng = fng.set_index('timestamp', drop=False)

# fng_upsampled = fng.resample('1h').ffill()

# # Copy the index to the timestamp column
# fng_upsampled['timestamp'] = fng_upsampled.index
# fng_upsampled

In [ ]:
# # Plot the data
# import matplotlib.pyplot as plt

# plt.figure(figsize=(14, 7))
# plt.plot(fng_upsampled['timestamp'], fng_upsampled['fng_value'], label='Fear and Greed Index')
# plt.title('Fear and Greed Index')
# plt.xlabel('Date')
# plt.ylabel('Fear and Greed Index')
# plt.legend()
# plt.grid()
# plt.show()

In [ ]:
# # Save the data to a CSV file
# output_file = os.path.join(ROOT, INDEXES_DATASET_PATH, 'fng_hourly.csv')
# fng_upsampled.to_csv(output_file, index=False)

# CBBI

In [28]:
url = "https://colintalkscrypto.com/cbbi/data/latest.json"
user_agent = "curl/8.4.0"
headers = {"User-Agent": user_agent}
response = requests.get(url, headers=headers)

cbbi_raw = None

# Check for a successful response
if response.status_code == 200:
    # Extract the JSON data
    data = response.json()
    
    # Extract the data from the JSON
    cbbi_raw = data
else:
    print("Errore nella richiesta all'API:", response.status_code)

# Extract the confidence data
confidence_data = data.get("Confidence", {}) # timestamp, value
cbbi_raw = pd.DataFrame(confidence_data, index=[0]) # Create a DataFrame from the confidence data

In [30]:
cbbi_raw

,1309132800,1309219200,1309305600,1309392000,1309478400,1309564800,1309651200,1309737600,1309824000,1309910400,...,1721606400,1721692800,1721779200,1721865600,1721952000,1722038400,1722124800,1722211200,1722297600,1722384000
0,0.6813,0.7058,0.7049,0.6921,0.6751,0.6705,0.6656,0.6495,0.6213,0.6251,...,0.7022,0.6883,0.6705,0.6856,0.6901,0.6961,0.6922,0.7039,0.6809,0.6792


In [31]:
cbbi = cbbi_raw.copy()

cbbi = cbbi.T # Transpose the DataFrame
cbbi.columns = ["cbbi_value"] # Rename the column to "value"
cbbi.index.name = "timestamp" # Rename the index to "timestamp"
cbbi.index = pd.to_datetime(cbbi.index, unit="s") # Convert the index to a datetime
cbbi = cbbi.sort_index() # Sort the DataFrame by the index
cbbi = cbbi.reset_index() # Reset the index
cbbi["cbbi_value"] = cbbi["cbbi_value"].astype(float).round(2) # Convert the value column to float and round to 2 decimal places
cbbi

C:\Users\danil\AppData\Local\Temp\ipykernel_38432\1948473157.py:6: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  cbbi.index = pd.to_datetime(cbbi.index, unit="s") # Convert the index to a datetime


,timestamp,cbbi_value
0,2011-06-27,0.68
1,2011-06-28,0.71
2,2011-06-29,0.70
3,2011-06-30,0.69
4,2011-07-01,0.68
...,...,...
4779,2024-07-27,0.70
4780,2024-07-28,0.69
4781,2024-07-29,0.70
4782,2024-07-30,0.68


In [32]:
# For each cbbi_value, use the following mapping to determine the sentiment
# cbbi_value < 0.4: negative
# 0.4 <= cbbi_value < 0.6: neutral
# cbbi_value >= 0.6: positive

cbbi['cbbi_sentiment'] = 'negative'
cbbi.loc[cbbi['cbbi_value'] >= 0.4, 'cbbi_sentiment'] = 'neutral'
cbbi.loc[cbbi['cbbi_value'] >= 0.6, 'cbbi_sentiment'] = 'positive'
cbbi


,timestamp,cbbi_value,cbbi_sentiment
0,2011-06-27,0.68,positive
1,2011-06-28,0.71,positive
2,2011-06-29,0.70,positive
3,2011-06-30,0.69,positive
4,2011-07-01,0.68,positive
...,...,...,...
4779,2024-07-27,0.70,positive
4780,2024-07-28,0.69,positive
4781,2024-07-29,0.70,positive
4782,2024-07-30,0.68,positive


In [33]:
# Trim the data to the specified date range
cbbi = cbbi[(cbbi["timestamp"] >= start_date) & (cbbi["timestamp"] <= end_date)]
cbbi

,timestamp,cbbi_value,cbbi_sentiment
1649,2016-01-01,0.29,negative
1650,2016-01-02,0.30,negative
1651,2016-01-03,0.30,negative
1652,2016-01-04,0.30,negative
1653,2016-01-05,0.30,negative
...,...,...,...
4779,2024-07-27,0.70,positive
4780,2024-07-28,0.69,positive
4781,2024-07-29,0.70,positive
4782,2024-07-30,0.68,positive


In [34]:
# Save the data to a CSV file
output_file = os.path.join(ROOT, INDEXES_DATASET_PATH, 'cbbi_daily.csv')
cbbi.to_csv(output_file, index=False)

In [ ]:
# # Set timestamp as the index
# cbbi = cbbi.set_index('timestamp', drop=False)

# # Upsampling to 1h by interpolate
# cbbi_upsampled = cbbi.resample('1h').ffill()

# # Copy the index to the timestamp column
# cbbi_upsampled['timestamp'] = cbbi_upsampled.index
# cbbi_upsampled

In [ ]:
# # Plot the data
# plt.figure(figsize=(14, 7))
# plt.plot(cbbi_upsampled['timestamp'], cbbi_upsampled['cbbi_value'], label='Crypto and Bitcoin Bullish Index')
# plt.title('Crypto and Bitcoin Bullish Index')
# plt.xlabel('Date')
# plt.ylabel('Crypto and Bitcoin Bullish Index')
# plt.legend()
# plt.grid()
# plt.show()

In [ ]:
# # Save the data to a CSV file
# output_file = os.path.join(ROOT, INDEXES_DATASET_PATH, 'cbbi_hourly.csv')
# cbbi_upsampled.to_csv(output_file, index=False)

# Merge data

In [35]:
# Load data
fng_daily = pd.read_csv(os.path.join(ROOT, INDEXES_DATASET_PATH, 'fng_daily.csv'))
cbbi_daily = pd.read_csv(os.path.join(ROOT, INDEXES_DATASET_PATH, 'cbbi_daily.csv'))

# fng_hourly = pd.read_csv(os.path.join(ROOT, INDEXES_DATASET_PATH, 'fng_hourly.csv'))
# cbbi_hourly = pd.read_csv(os.path.join(ROOT, INDEXES_DATASET_PATH, 'cbbi_hourly.csv'))

In [41]:
# Return the dataset that have the minimum date
min_date = min(fng_daily['timestamp'].min(), cbbi_daily['timestamp'].min())

if min_date == fng_daily['timestamp'].min():
    print("The dataset with the minimum date is fng_daily") 
elif min_date == cbbi_daily['timestamp'].min():
    print("The dataset with the minimum date is cbbi_daily")
else:
    print("The minimum date is the same for both datasets")

The dataset with the minimum date is cbbi_daily


In [44]:
# Merge data from
merged_daily = pd.merge(fng_daily, cbbi_daily, how="right", on='timestamp')
# merged_hourly = pd.merge(fng_hourly, cbbi_hourly, how="inner", on='timestamp')

In [45]:
merged_daily

,fng_value,fng_value_classification,timestamp,fng_sentiment,cbbi_value,cbbi_sentiment
0,NaN,NaN,2016-01-01,NaN,0.29,negative
1,NaN,NaN,2016-01-02,NaN,0.30,negative
2,NaN,NaN,2016-01-03,NaN,0.30,negative
3,NaN,NaN,2016-01-04,NaN,0.30,negative
4,NaN,NaN,2016-01-05,NaN,0.30,negative
...,...,...,...,...,...,...
3130,0.72,3.0,2024-07-27,positive,0.70,positive
3131,0.71,3.0,2024-07-28,positive,0.69,positive
3132,0.74,3.0,2024-07-29,positive,0.70,positive
3133,0.67,3.0,2024-07-30,positive,0.68,positive


In [ ]:
# merged_hourly

In [46]:
# Save the data to a CSV file
output_file = os.path.join(ROOT, INDEXES_DATASET_PATH, 'indexes_daily.csv')
merged_daily.to_csv(output_file, index=False)

# Save the data to a CSV file
# output_file = os.path.join(ROOT, INDEXES_DATASET_PATH, 'indexes_hourly.csv')
# merged_hourly.to_csv(output_file, index=False)